In [4]:
from utils import *
from gtfs_realtime_utils import *
import shapely

In [ ]:
DATADIR = ROOT / "data/geojson/north_west/real"
fnames = os.listdir(DATADIR)
# gdfs = [gpd.read_file(DATADIR / f) for f in fnames]
# gdfs

In [ ]:
valid_geometries = []
for f in fnames:
    gdf = gpd.read_file(DATADIR / f)
    if gdf.is_valid[0] == True:
        valid_geometry = gdf['geometry']
    else:
        valid_geometry = shapely.make_valid(gdf['geometry'])
        valid_geometry = valid_geometry.set_crs('EPSG:4326')
        # print(valid_geometry.crs)
    valid_geometries.append(valid_geometry)
valid_geometries

In [10]:
result = valid_geometries[0].intersection(valid_geometries[1]).intersection(valid_geometries[2]).intersection(valid_geometries[3]).intersection(valid_geometries[4])
result = result.explode()

In [11]:
result.to_file(DATADIR / "20240916_17:00:00_intersection.geojson", driver='GeoJSON')

In [ ]:
gpd.GeoSeries(shapely.Point((-2.981029, 53.40592))).to_file(DATADIR / 'origin.geojson', driver='GeoJSON')